In [1]:
import gc
import os
import torch
import itertools
import numpy as np
import torch.nn as nn
import matplotlib as plt
import torch.optim as optim
import matplotlib.pyplot as plt
from kan.MultKAN import MultKAN
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score

In [2]:
DATA_type = 'mels_kan_1'
Model_name = 'kan_mels_1'
Train_data = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Train_data.pt')
Train_label = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Train_label.pt')
Val_data = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Val_data.pt')
Val_label = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Val_label.pt')
Test_data = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Test_data.pt')
Test_label = torch.load(f'../make_stored_data/sliding_window/{DATA_type}/Test_label.pt')
print(np.shape(Train_data))
print(np.shape(Train_label))
print(np.shape(Val_data))
print(np.shape(Val_label))
print(np.shape(Test_data))
print(np.shape(Test_label))

torch.Size([5, 7200, 256])
torch.Size([5, 7200])
torch.Size([5, 1800, 256])
torch.Size([5, 1800])
torch.Size([1000, 256])
torch.Size([1000])


In [3]:
# class F2Score(tf.keras.metrics.Metric):
#     def __init__(self, name='f2_score', threshold=0.3, **kwargs):
#         super(F2Score, self).__init__(name=name, **kwargs)
#         self.threshold = threshold
#         self.tp = self.add_weight(name='tp', initializer='zeros')
#         self.fp = self.add_weight(name='fp', initializer='zeros')
#         self.fn = self.add_weight(name='fn', initializer='zeros')

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.cast(y_pred > self.threshold, tf.float32)
#         y_true = tf.cast(y_true, tf.float32)
#         self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
#         self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
#         self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

#     def result(self):
#         precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
#         recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
#         return 5 * (precision * recall) / (4 * precision + recall + tf.keras.backend.epsilon())

#     def reset_state(self):
#         self.tp.assign(0)
#         self.fp.assign(0)
#         self.fn.assign(0)

# class F2ScoreLogger(Callback):
#     def __init__(self, validation_data, threshold=0.3):
#         super().__init__()
#         self.X_val, self.y_val = validation_data
#         self.threshold = threshold
#         self.y_preds = []
#         self.f2_scores = []
#         self.recalls = []
#         self.precisions = []
#         self.best_weights = None
#         self.best_f2 = 0

#     def on_epoch_end(self, epoch, logs=None):
#         y_pred_prob = self.model.predict(self.X_val, verbose=0)
#         y_pred = (y_pred_prob > self.threshold).astype('int32')
#         recall = recall_score(self.y_val, y_pred)
#         precision = precision_score(self.y_val, y_pred)
#         f2 = 5 / ((4 / recall) + (1 / precision)) if recall and precision else 0

#         self.y_preds.append(y_pred)
#         self.recalls.append(recall)
#         self.precisions.append(precision)
#         self.f2_scores.append(f2)

#         if f2 > self.best_f2:
#             self.best_f2 = f2
#             self.best_weights = self.model.get_weights()

#     def on_train_end(self, logs=None):
#         if self.best_weights is not None:self.model.set_weights(self.best_weights)

In [4]:
# batch_sizes = [16]
# learning_rates = [0.0001, 0.0005]
# hidden_widths = [64, 128]
# grids = [3, 5, 10]

# param_combinations = list(itertools.product(batch_sizes, learning_rates, hidden_widths, grids))

# results = []
# best_model = None
# best_model_config_id = None
# best_model_f2 = 0

# for i, (batch_size, lr, width, grid) in enumerate(param_combinations):
#     fold_recalls = []
#     fold_f2_scores = []
#     fold_histories = []
#     fold_conf_matrices = []

#     for fold in range(5):
#         best_f2 = 0
#         print(f"Running config {i+1}/{len(param_combinations)}")
#         print("Config:")
#         print(f"  Batch Size     : {batch_size}")
#         print(f"  Learning Rate  : {lr}")
#         print(f"  Hidden Width   : {width}")
#         print(f"  Grid           : {grid}")
#         print("-" * 40)
#         print(f"Fold : {fold+1}")

#         X_train, X_val = Train_data[fold], Val_data[fold]
#         y_train, y_val = Train_label[fold], Val_label[fold]

#         model = MultKAN(width=[1024, width, 1], grid=grid, k=3, device="cuda" if torch.cuda.is_available() else "cpu")
        
#         optimizer = optim.Adam(model.parameters(), lr=lr)
#         criterion = nn.BCEWithLogitsLoss()

#         for epoch in range(100):
#             model.train()
#             perm = torch.randperm(X_train.size(0))
#             for step in range(0, X_train.size(0), batch_size):
#                 indices = perm[step:step + batch_size]
#                 X_batch = X_train[indices]
#                 y_batch = y_train[indices]

#                 optimizer.zero_grad()
#                 y_pred = model(X_batch)
#                 y_batch = y_batch.view(-1, 1).float()
#                 loss = criterion(y_pred, y_batch)
#                 loss.backward()
#                 optimizer.step()
#                 print('complete')

#             model.eval()
#             with torch.no_grad():
#                 y_pred = model(X_val).cpu().numpy()
#                 y_pred_bin = (y_pred > 0.3).astype("int32")

#             y_true = y_val.cpu().numpy().astype("int32")

#             recall = recall_score(y_true, y_pred_bin)
#             precision = precision_score(y_true, y_pred_bin)
#             f2 = 5 / ((4 / recall) + (1 / precision)) if recall and precision else 0
            
#             if f2 > best_f2:
#                 best_f2 = f2
#                 best_epoch_weights = model.state_dict()
#                 best_epoch = epoch
#             print(f'epoch {epoch} completed')

#         model.eval()
#         with torch.no_grad():
#             fold_y_pred = model(X_val).cpu().numpy()
#             fold_y_pred_bin = (fold_y_pred > 0.3).astype("int32")
            
#         fold_y_true = y_val.cpu().numpy().astype("int32")

#         fold_recall = recall_score(fold_y_true, fold_y_pred_bin)
#         fold_precision = precision_score(fold_y_true, fold_y_pred_bin)
#         fold_f2 = 5 / ((4 / fold_recall) + (1 / fold_precision)) if fold_recall and fold_precision else 0
#         cm = confusion_matrix(fold_y_true, fold_y_pred_bin)

#         fold_recalls.append(fold_recall)
#         fold_f2_scores.append(fold_f2)
#         fold_histories.append(history)
#         fold_conf_matrices.append((cm, fold_y_true, fold_y_pred_bin))
        
#         clear_output(wait=False)

#     best_fold = int(np.argmax(fold_f2_scores))
#     avg_f2 = np.mean(fold_f2_scores)
#     avg_recall = np.mean(fold_recalls)

#     best_history = fold_histories[best_fold]
#     best_cm, best_y_val, best_y_pred = fold_conf_matrices[best_fold]

#     results.append({
#         'config_id': i,
#         'batch_size': batch_size,
#         'learning_rate': lr,
#         'hidden_size': hidden_size,
#         'dropout': dropout_rate,
#         'kernel_reg': kernel_reg,
#         'avg_val_recall': avg_recall,
#         'best_recall': fold_recalls[best_fold],
#         'avg_val_f2': avg_f2,
#         'best_f2': fold_f2_scores[best_fold],
#         'history': best_history,
#         'confusion_matrix': best_cm
#     })

#     if fold_f2_scores[best_fold] > best_model_f2:
#         best_model_f2 = fold_f2_scores[best_fold]
#         best_model_config_id = i
#         best_model = {
#             'history': best_history,
#             'confusion_matrix': best_cm,
#         }

In [3]:
grids = [2, 3, 4]
lambs = [0.001, 0.002, 0.003]

param_combinations = list(itertools.product(grids, lambs))

results = []
best_model = None
best_model_config_id = None
best_model_f2 = 0

for i, (grid, lamb) in enumerate(param_combinations):
    fold_recalls = []
    fold_f2_scores = []
    fold_histories = []
    fold_conf_matrices = []

    for fold in range(5):
        best_f2 = 0
        print(f"Running config {i+1}/{len(param_combinations)}")
        print("Config:")
        print(f"  Grid   : {grid}")
        print(f"  Lamb   : {lamb}")
        print("-" * 40)
        print(f"Fold : {fold+1}")

        X_train, X_val = Train_data[fold], Val_data[fold]
        y_train, y_val = Train_label[fold], Val_label[fold]

        dataset = {'train_input': X_train, 'train_label': y_train, 'test_input': X_val, 'test_label': y_val}

        model = MultKAN(width=[256, 5, 1], grid=grid, k=3, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # optimizer = optim.Adam(model.parameters(), lr=lr)
        # criterion = nn.BCEWithLogitsLoss()

        model.train()
        model.fit(dataset, opt="LBFGS", steps=50, lamb=lamb)
        model.eval()
        
        with torch.no_grad():
            fold_y_pred = model(X_val).cpu().numpy()
            fold_y_pred_bin = (fold_y_pred > 0.3).astype("int32")
            
        fold_y_true = y_val.cpu().numpy().astype("int32")

        fold_recall = recall_score(fold_y_true, fold_y_pred_bin)
        fold_precision = precision_score(fold_y_true, fold_y_pred_bin)
        fold_f2 = 5 / ((4 / fold_recall) + (1 / fold_precision)) if fold_recall and fold_precision else 0
        cm = confusion_matrix(fold_y_true, fold_y_pred_bin)

        fold_recalls.append(fold_recall)
        fold_f2_scores.append(fold_f2)
        fold_histories.append(history)
        fold_conf_matrices.append((cm, fold_y_true, fold_y_pred_bin))
        
        clear_output(wait=False)

    best_fold = int(np.argmax(fold_f2_scores))
    avg_f2 = np.mean(fold_f2_scores)
    avg_recall = np.mean(fold_recalls)

    best_history = fold_histories[best_fold]
    best_cm, best_y_val, best_y_pred = fold_conf_matrices[best_fold]

    results.append({
        'config_id': i,
        'batch_size': batch_size,
        'learning_rate': lr,
        'hidden_size': hidden_size,
        'dropout': dropout_rate,
        'kernel_reg': kernel_reg,
        'avg_val_recall': avg_recall,
        'best_recall': fold_recalls[best_fold],
        'avg_val_f2': avg_f2,
        'best_f2': fold_f2_scores[best_fold],
        'history': best_history,
        'confusion_matrix': best_cm
    })

    if fold_f2_scores[best_fold] > best_model_f2:
        best_model_f2 = fold_f2_scores[best_fold]
        best_model_config_id = i
        best_model = {
            'history': best_history,
            'confusion_matrix': best_cm,
        }

Running config 1/9
Config:
  Grid   : 2
  Lamb   : 0.001
----------------------------------------
Fold : 1
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 5.00e-01 | test_loss: 5.00e-01 | reg: 7.55e-01 | :  19%|▏| 19/100 [04:37<19:43, 14.61s


KeyboardInterrupt: 

In [ ]:
best_result = results[best_model_config_id]

print("Best Model Metric")
print("-" * 40)
print(f"Config ID             : {best_model_config_id}")
print(f"Batch Size            : {best_result['batch_size']}")
print(f"Learning Rate         : {best_result['learning_rate']}")
print(f"Hidden Units          : {best_result['hidden_size']}")
print(f"Dropout Rate          : {best_result['dropout']}")
print(f"Kernel Reg L2         : {best_result['kernel_reg']}")
print(f"Avg Recall (5-fold)   : {best_result['avg_val_recall']:.4f}")
print(f"Best Fold Recall      : {best_result['best_recall']:.4f}")
print(f"Avg F2-score (5-fold) : {best_result['avg_val_f2']:.4f}")
print(f"Best Fold F2-score    : {best_result['best_f2']:.4f}")
print("-" * 40)

best_history = best_model['history']
best_cm = best_model['confusion_matrix']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].plot(best_history.history['loss'], label='Training Loss')
if 'val_loss' in best_history.history:
    axes[0, 0].plot(best_history.history['val_loss'], label='Validation Loss')
axes[0, 0].set_title('Model Loss')
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()

if 'f2_score' in best_history.history:
    axes[0, 1].plot(best_history.history['f2_score'], label='Training F2 Score')
if 'val_f2_score' in best_history.history:
    axes[0, 1].plot(best_history.history['val_f2_score'], label='Validation F2 Score')
axes[0, 1].set_title('Model F2 Score')
axes[0, 1].set_xlabel('Epochs')
axes[0, 1].set_ylabel('F2 Score')
axes[0, 1].legend()

if 'lr' in best_history.history:
    axes[1, 0].plot(best_history.history['lr'], label='Learning Rate')
elif 'learning_rate' in best_history.history:
    axes[1, 0].plot(best_history.history['learning_rate'], label='Learning Rate')
axes[1, 0].set_title('Learning Rate')
axes[1, 0].set_xlabel('Epochs')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].legend()

disp = ConfusionMatrixDisplay(confusion_matrix=best_cm, display_labels=["Normal", "Abnormal"])
disp.plot(cmap=plt.cm.Blues, ax=axes[1, 1])
axes[1, 1].set_title("Best Model Confusion Matrix (Validation)")

plt.tight_layout()
plt.show()

In [ ]:
model.set_weights(f2_logger.best_weights)
Test_pred = (model.predict(Test_data) > 0.3).astype("int32")

Test_acc = accuracy_score(Test_label, Test_pred)
Test_precision = precision_score(Test_label, Test_pred)
Test_recall = recall_score(Test_label, Test_pred)
Test_f2 = 5 / ((4 / Test_recall) + (1 / Test_precision)) if Test_recall and Test_precision else 0
Test_cm = confusion_matrix(Test_label, Test_pred)

print("=== Test Set Performance ===")
print(f"Accuracy  : {Test_acc:.4f}")
print(f"Precision : {Test_precision:.4f}")
print(f"Recall    : {Test_recall:.4f}")
print(f"F2 Score  : {Test_f2:.4f}")
print("Confusion Matrix:")

Test_disp = ConfusionMatrixDisplay(confusion_matrix=Test_cm, display_labels=["Normal", "Abnormal"])
Test_disp.plot(cmap=plt.cm.Blues)
plt.title("Best Model Confusion Matrix (Test)")
plt.tight_layout()
plt.show()

In [ ]:
model.save(f'../new_model/{Model_name}.h5')